In [10]:
import os
import random

import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from optuna import trial
from sentry_sdk.utils import epoch
from sklearn.model_selection import train_test_split
from torchvision import transforms, models
from torchvision.transforms.functional import to_pil_image
from torch.utils.data import Dataset, DataLoader, Subset, ConcatDataset

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import VGG19_Weights
from torchvision.transforms.v2 import Transform

import optuna
import wandb

# ***CHANGE to your directory if not working locally***
# DIR = r'C:\Users\amitr5\PycharmProjects\deep_van_gogh'
DIR = r'E:\My Drive\Intro to deep learning\deep_van_gogh'
os.chdir(DIR)
print("Current Working Directory:", os.getcwd())

# Project utilities
import utils
import preprocessing

# Set seed
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

np.random.seed(SEED)
random.seed(SEED)
# torch.backends.cudnn.deterministic = True
# torch.use_deterministic_algorithms = True

ROOT = './data/dataset'
DATASET_DIR = 'data/dataset/Post_Impressionism'
CSV_PATH = 'data/dataset/classes.csv'
OPTIMIZED_DIR = './data/optimized'

if not (os.path.exists(DATASET_DIR) and os.path.exists(CSV_PATH)):
    raise FileExistsError("File doesn't exist, we expect a data folder with the labels and the images folder as in the lab")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Current Working Directory: E:\My Drive\Intro to deep learning\deep_van_gogh


In [31]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(), # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229,0.224, 0.225])]) # Normalize the pixel values based on ImageNet statistics, to a range that VGG expects

optimized_dataset_path = OPTIMIZED_DIR + '/dataset.npz'

if os.path.exists(optimized_dataset_path):
    dataset = preprocessing.NumPyDataset('data/optimized/dataset.npz')
    pics_in_csv = {i for i in range(len(dataset)) if dataset.labels[i] >= 0}
else:
    dataset = preprocessing.ImageFolderForBinaryClassification(root=ROOT, target='is_van_gogh', transform=transform)
    pics_in_csv = {i for i in range(len(dataset)) if dataset.samples[i][1] >= 0}

# Take only pictures that are in inside the csv
classes = pd.read_csv(CSV_PATH)
train_classes = list(pics_in_csv.intersection(classes[classes['subset'] == 'train'].index.tolist()))
test_classes = list(pics_in_csv.intersection(classes[classes['subset'] == 'test'].index.tolist()))

In [32]:
train_indices, val_indices = train_test_split(list(train_classes), test_size=0.2, random_state=SEED)
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)

### Data Augmentation

In [38]:
torch.manual_seed(SEED)
n_times = 25
dropout_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    *([transforms.RandomErasing(p=0.5, scale=(0.01, 0.01), ratio=(1, 1))]*n_times),
    transforms.Grayscale(num_output_channels=3),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229,0.224, 0.225])
])
dropout_dataset = Subset(preprocessing.ImageFolderForBinaryClassification(root=ROOT, transform=dropout_transform, target='is_van_gogh'), train_indices)
# augmented_train_dataset = ConcatDataset([dropout_dataset, train_dataset])
augmented_train_dataset = train_dataset
# train_loader = DataLoader(augmented_train_dataset, batch_size=128, shuffle=True, num_workers=4, pin_memory=True, prefetch_factor=8)
# val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4)

# Fine tuning VGG19

In [35]:
# Load pre-trained VGG19 model
vgg_model = models.vgg19(weights=models.VGG19_Weights.DEFAULT).to(device)
for param in vgg_model.features.parameters():
        param.requires_grad = False

vgg_model.classifier[-1] = nn.Linear(4096, 2).to(device)
vgg_model


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [39]:
from sklearn.metrics import roc_auc_score
from train import train_model_with_hyperparams

# Optuna objective function
def objective(trial, model):
    # Hyperparameter suggestions
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3) #Suggests a learning_rate value from a log-uniform distribution between 1e-5 and 1e-3 for hyperparameter optimization using Optuna.
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-6, 1e-4) # Suggests a weight_decay value from a log-uniform distribution between 1e-6 and 1e-4 for regularization during Optuna hyperparameter optimization.
    patience = trial.suggest_int("patience", 3, 10) #I don't really like putting the patience as a hyperparameter - this is a thing that needs to be determined according to constraints.
    batch_size = trial.suggest_int("batch_size", 16, 64, step=16) # Basically choosing between 16,32,64

    train_loader = DataLoader(augmented_train_dataset, batch_size=batch_size, shuffle=True) # Load the train DataLoader with the chosen batch_size
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False) # Load the val DataLoader with the chosen batch_size

    # Load the pre-trained model VGG16
    # model = models.vgg19(weights=models.VGG19_Weights.DEFAULT).to(device)

    # Freeze layers
    for param in model.features.parameters():
        param.requires_grad = False

    # Unfreeze the last 6 layers in the features part of the VGG
    # for param in model.features[-6:].parameters():
    #     param.requires_grad = True

    # Modify the classifier to fit our problem (10 classes)
    model.classifier[-1] = nn.Linear(4096, 2).to(device) # Replaces the final layer of the VGG16 classifier with a new fully connected (nn.Linear) layer. The new wights are initialized
    # Define optimizer and loss function
    criterion = nn.CrossEntropyLoss() # Classification.
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay) # Adam, like always, with the chosen parameters from Optuna

    # Initialize Weights & Biases - the values in the config are the properties of each trial.
    wandb.init(project="deep_van_gogh-vgg19", #init == set the project and the "general" parameters
               config={
        "learning_rate": learning_rate,
        "weight_decay": weight_decay,
        "patience": patience,
        "batch_size": batch_size,
        "architecture": "VGG19",
        "dataset": "Post_Imp-10"
    },
    name=f"trial_{trial.number}") # The name that will be saved in the W&B platform

    # Train the model and get the best validation loss
    epochs = 5
    best_val_loss = train_model_with_hyperparams(model, train_loader, val_loader, optimizer, criterion, epochs=epochs, patience=patience, trial=trial, device=device) #send this trinal to the function above

    # Finish the Weights & Biases run
    wandb.finish()

    # Return best validation loss as the objective to minimize
    return best_val_loss

In [40]:
study = optuna.create_study(direction='minimize')
study.optimize(lambda trial : objective(trial, vgg_model), n_trials=1, )

[I 2025-01-14 14:07:26,307] A new study created in memory with name: no-name-d922a91e-1a0e-4bdb-81dd-8411df0cb0c0
C:\Users\Amit\AppData\Local\Temp\ipykernel_27776\202831435.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3) #Suggests a learning_rate value from a log-uniform distribution between 1e-5 and 1e-3 for hyperparameter optimization using Optuna.
C:\Users\Amit\AppData\Local\Temp\ipykernel_27776\202831435.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-6, 1e-4) # Suggests a weight_decay value from a log-uniform distribution between 

Epoch,▁▃▅▆█
Train Accuracy,▁▃▆██
Train Loss,█▅▂▁▁
Validation Accuracy,▁▆▅█▇
Validation Loss,▅▁▅██
Epoch,5
Train Accuracy,0.99956
Train Loss,0.00141
Validation Accuracy,0.97378
Validation Loss,0.18514


[I 2025-01-14 14:08:56,766] Trial 0 finished with value: 0.09964846864251511 and parameters: {'learning_rate': 7.732642367785799e-05, 'weight_decay': 9.214387766525893e-06, 'patience': 7, 'batch_size': 16}. Best is trial 0 with value: 0.09964846864251511.


# Fine tuning AlexNet

In [ ]:
# Load the AlexNet model 
alexnet = models.alexnet(pretrained=True)


analysing results

# Style transfer function